In [11]:
import pandas as pd

orders = pd.read_csv(r"C:\Users\SIGMA\Downloads\orders.csv")
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [12]:
users = pd.read_json(r"C:\Users\SIGMA\Downloads\users.json")
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [8]:
import sqlite3
conn = sqlite3.connect("restaurants.db")
cursor = conn.cursor()   

with open(r"C:\Users\SIGMA\Downloads\restaurants.sql", "r", encoding="utf-8") as f:
    sql_script = f.read()

cursor.executescript(sql_script)
conn.commit()

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

print(restaurants.head())


   restaurant_id restaurant_name  cuisine  rating
0              1    Restaurant_1  Chinese     4.8
1              2    Restaurant_2   Indian     4.1
2              3    Restaurant_3  Mexican     4.3
3              4    Restaurant_4  Chinese     4.1
4              5    Restaurant_5  Chinese     4.8


In [9]:
orders_users = orders.merge(
    users,
    on="user_id",
    how="left"
)


In [10]:
final_df = orders_users.merge(
    restaurants,
    on="restaurant_id",
    how="left"
)


In [13]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


In [17]:
df=pd.read_csv("final_food_delivery_dataset.csv")

In [18]:
df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [19]:
df[df["membership"] == "Gold"] \
    .groupby("city")["total_amount"] \
    .sum() \
    .sort_values(ascending=False) \
    .head(1)


city
Chennai    1080909.79
Name: total_amount, dtype: float64

In [20]:
df.groupby("cuisine")["total_amount"] \
  .mean() \
  .sort_values(ascending=False) \
  .head(1)


cuisine
Mexican    808.021344
Name: total_amount, dtype: float64

In [21]:
user_spend = df.groupby("user_id")["total_amount"].sum()
user_spend[user_spend > 1000].count()


np.int64(2544)

In [22]:
df["rating_range"] = pd.cut(
    df["rating"],
    bins=[3.0, 3.5, 4.0, 4.5, 5.0],
    labels=["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]
)

df.groupby("rating_range")["total_amount"] \
  .sum() \
  .sort_values(ascending=False)


C:\Users\SIGMA\AppData\Local\Temp\ipykernel_7252\2937914223.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("rating_range")["total_amount"] \


rating_range
4.6–5.0    2197030.75
4.1–4.5    1960326.26
3.0–3.5    1881754.57
3.6–4.0    1717494.41
Name: total_amount, dtype: float64

In [23]:
df[df["membership"] == "Gold"] \
    .groupby("city")["total_amount"] \
    .mean() \
    .sort_values(ascending=False) \
    .head(1)


city
Chennai    808.45908
Name: total_amount, dtype: float64

In [24]:
restaurant_count = df.groupby("cuisine")["restaurant_id"].nunique()
revenue = df.groupby("cuisine")["total_amount"].sum()

result = pd.concat([restaurant_count, revenue], axis=1)
result.columns = ["restaurant_count", "revenue"]
result.sort_values("restaurant_count")


,restaurant_count,revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [25]:
gold_orders = df[df["membership"] == "Gold"].shape[0]
total_orders = df.shape[0]

round((gold_orders / total_orders) * 100)


50

In [31]:
restaurant_stats = (
    df.groupby("restaurant_name_y")
      .agg(
          avg_order_value=("total_amount", "mean"),
          order_count=("order_id", "count")
      )
)

restaurant_stats[
    restaurant_stats["order_count"] < 20
].sort_values(
    by="avg_order_value",
    ascending=False
).head(1)


,avg_order_value,order_count
restaurant_name_y,,
Restaurant_294,1040.222308,13


In [32]:
df.groupby(["membership", "cuisine"])["total_amount"] \
  .sum() \
  .sort_values(ascending=False) \
  .head(1)

membership  cuisine
Regular     Mexican    1072943.3
Name: total_amount, dtype: float64

In [36]:

filtered_df = df[
    ((df["membership"] == "Gold") & (df["cuisine"] == "Indian")) |
    ((df["membership"] == "Gold") & (df["cuisine"] == "Italian")) |
    ((df["membership"] == "Regular") & (df["cuisine"] == "Indian")) |
    ((df["membership"] == "Regular") & (df["cuisine"] == "Chinese"))
]
combo_revenue = (
    filtered_df
    .groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .reset_index()
)

highest_combo = combo_revenue.loc[
    combo_revenue["total_amount"].idxmax()
]

print(combo_revenue)
print("\nHighest Revenue Combination:")
print(highest_combo)

  membership  cuisine  total_amount
0       Gold   Indian     979312.31
1       Gold  Italian    1005779.05
2    Regular  Chinese     952790.91
3    Regular   Indian     992100.27

Highest Revenue Combination:
membership            Gold
cuisine            Italian
total_amount    1005779.05
Name: 1, dtype: object


In [35]:

df["order_date"] = pd.to_datetime(df["order_date"])
df["quarter"] = df["order_date"].dt.month.map(
    lambda x: "Q1 (Jan–Mar)" if x in [1, 2, 3] else
              "Q2 (Apr–Jun)" if x in [4, 5, 6] else
              "Q3 (Jul–Sep)" if x in [7, 8, 9] else
              "Q4 (Oct–Dec)"
)

quarter_revenue = (
    df.groupby("quarter")["total_amount"]
    .sum()
    .reset_index()
)

highest_quarter = quarter_revenue.loc[
    quarter_revenue["total_amount"].idxmax()
]

print(quarter_revenue)
print("\nHighest Revenue Quarter:")
print(highest_quarter)

        quarter  total_amount
0  Q1 (Jan–Mar)    2010626.64
1  Q2 (Apr–Jun)    1945348.72
2  Q3 (Jul–Sep)    2037385.10
3  Q4 (Oct–Dec)    2018263.66

Highest Revenue Quarter:
quarter         Q3 (Jul–Sep)
total_amount       2037385.1
Name: 2, dtype: object


C:\Users\SIGMA\AppData\Local\Temp\ipykernel_7252\934070039.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["order_date"] = pd.to_datetime(df["order_date"])


In [41]:
df["order_date"] = pd.to_datetime(df["order_date"])
gold_orders_count = df[df["membership"] == "Gold"].shape[0]
print("Total Gold member orders:", gold_orders_count)

Total Gold member orders: 4987


In [42]:

hyderabad_revenue = round(
    df[df["city"] == "Hyderabad"]["total_amount"].sum())
print("Hyderabad total revenue:", hyderabad_revenue)

Hyderabad total revenue: 1889367


In [43]:
distinct_users = df["user_id"].nunique()
print("Distinct users:", distinct_users)

Distinct users: 2883


In [44]:
avg_order_value_gold = round(
    df[df["membership"] == "Gold"]["total_amount"].mean(), 2
)
print("Avg order value (Gold):", avg_order_value_gold)

Avg order value (Gold): 797.15


In [45]:
high_rating_orders = df[df["rating"] >= 4.5].shape[0]
print("Orders with rating ≥ 4.5:", high_rating_orders)

Orders with rating ≥ 4.5: 3374


In [46]:
gold_df = df[df["membership"] == "Gold"]
top_city = (
    gold_df.groupby("city")["total_amount"]
    .sum()
    .idxmax()
)
orders_top_city_gold = gold_df[gold_df["city"] == top_city].shape[0]
print("Orders in top Gold revenue city:", orders_top_city_gold)


Orders in top Gold revenue city: 1337


In [56]:
#The total number of rows in the final merged dataset is ____
len(data)

10000